In [ ]:
from rdflib import *
import json
from datetime import datetime, timedelta

class CommonCoreOntology():
    def __init__(self):
        self.cco = Graph().parse("../../../CommonCoreOntologies/src/cco-merged/CommonCoreOntologiesMerged.ttl")
        
        self.cco_ns = {x[0]:x[1] for x in self.cco.namespaces()}
        self.query_preamble = "\n".join([f"PREFIX {pfx}: <{self.cco_ns[pfx]}>" for pfx in self.cco_ns])
        
    def execute(self,querytext=""):
        q = "\n\n".join([self.query_preamble , querytext])
        start = datetime.now()
        res = self.cco.query(q)
        end = datetime.now()
        
        return {
            "result": {
                "query_duration_ms": round((end - start).microseconds / 1000,3), 
                "data": [c.asdict() for c in res]
            }
        }
    
    def execute_get_resultset(self,querytext=""):
        q = "\n\n".join([self.query_preamble , querytext])
        start = datetime.now()
        res = self.cco.query(q)
        end = datetime.now()
        
        #return {
        #    "result": {
        #        "query_duration_ms": round((end - start).microseconds / 1000,3), 
        #        "data": [c.asdict() for c in res]
        #    }
        #}   
        return {"results": res, "query_runtime_ms": round((end - start).microseconds / 1000,3)}
    
    
    #def get_count(self,count_query):
    #    return [r for r in res.query(count_query) ]

model = CommonCoreOntology() 

## Recursive queries to explore UnionOf, IntersectionOf

In [ ]:
dsq = "explore intersectionof components (lists) - and the recursiveness of SPARQL"

edges = """

SELECT *
WHERE {
  #?u owl:intersectionOf <http://purl.obolibrary.org/obo/BFO_0000030> .
  ?u owl:intersectionOf ?l .
  ?l rdf:rest* ?sl .   ## retrieve rest of ?l
  ?sl rdf:first ?e1 .  ## head of ?sl (first)
  ?sl rdf:rest ?sln .  ## 
  ?sln rdf:first ?e2 .
}
"""

data_returned =  model.execute(edges)
DSA = {
    #"count_result": count_result.get('result')
    "query": edges
    , "data_returned": data_returned.get('result')
    , "dsq": dsq
} 
open("dsq_results/Q7_intersection_of.json",'w').write(json.dumps(DSA,indent=2)) 

##print(json.dumps(DSA,indent=2))


In [ ]:
dsq = "explore intersectionof components (lists) - and the recursiveness of SPARQL"
edges = """

SELECT *
WHERE {
  ?u owl:unionOf ?l .
  ?l rdf:rest* ?sl .   ## retrieve rest of ?l
  ?sl rdf:first ?e1 .  ## head of ?sl (first)
  ?sl rdf:rest ?sln .  ## 
  ?sln rdf:first ?e2 .
}
"""

data_returned =  model.execute(edges)
DSA = {
    #"count_result": count_result.get('result')
    "query": edges
    , "data_returned": data_returned.get('result')
    , "dsq": dsq
} 
open("dsq_results/Q6_union_of.json",'w').write(json.dumps(DSA,indent=2)) 
print(json.dumps(DSA,indent=2))


In [ ]:
## locate the pairs of disjoint classes
dsq = ""
edges = """
SELECT DISTINCT *
WHERE  {    
    ?a owl:disjointWith ?b .
}
"""


data_returned =  model.execute(edges)
DSA = {
    "query": edges, 
    #"count_result": count_result.get('result')
     "data_returned": data_returned.get('result')
    , "dsq": dsq
}
print(json.dumps(DSA, indent=2))
open("dsq_results/Q8_disjoint_with.json",'w').write(json.dumps(DSA, indent=2))

In [ ]:
dsq = "How many axioms involve object properties on which both domain and range have also been asserted?"

count  = """
SELECT (COUNT(DISTINCT ?r) as ?restriction_count) 
WHERE {
    ?target_class a owl:Class .
    ##?property a owl:ObjectProperty .  ## drastically increases runtime
    
    ?r a owl:Restriction; 
        owl:onProperty ?property;
        owl:allValuesFrom ?target_class.
    ?property rdfs:label ?property_label.
    
    #OPTIONAL {
        ?property rdfs:domain ?domain; rdfs:range ?range.
        ?domain rdfs:label ?domain_class_label .
        ?range rdfs:label ?range_class_label .
    #}
    ?target_class rdfs:label ?target_class_label.
}
"""
count_result = model.execute(count) 


q  = """
SELECT DISTINCT ?property_label ?domain_class_label ?range_class_label ?target_class_label
WHERE {
    ?target_class a owl:Class .
    ##?property a owl:ObjectProperty .  ## drastically increases runtime
    
    ?r a owl:Restriction; 
        owl:onProperty ?property;
        owl:allValuesFrom ?target_class.
    ?property rdfs:label ?property_label.
    
    #OPTIONAL {
        ?property rdfs:domain ?domain; rdfs:range ?range.
        ?domain rdfs:label ?domain_class_label .
        ?range rdfs:label ?range_class_label .
    #}
    ?target_class rdfs:label ?target_class_label.
}
"""
data_returned =  model.execute(q)
DSA = {
    "count_result": count_result.get('result')
    , "data_returned": data_returned.get('result')
    , "dsq": dsq
}

    
##print(json.dumps(DSA       , indent=2))  

open('dsq_results/Q4.json','w').write(json.dumps(DSA       , indent=2))



In [ ]:
dsq = "How many axioms involve classes which have no URIs? "

count  = """
SELECT (COUNT(DISTINCT ?r) as ?restriction_count) 
WHERE {
    ?target_class a owl:Restriction .
    ##?property a owl:ObjectProperty .  ## drastically increases runtime
    
    ?r a owl:Restriction; 
        owl:onProperty ?property;
        owl:someValuesFrom ?target_class.

    ?property rdfs:label ?property_label.
    
    #OPTIONAL {
    #    ?property rdfs:domain ?domain; rdfs:range ?range.
    #    ?domain rdfs:label ?domain_class_label .
    #    ?range rdfs:label ?range_class_label .
    #}
    ?target_class rdfs:label ?target_class_label.
}
"""
count_result = model.execute(count) 


q  = """
SELECT DISTINCT ?p ?p_o ?oe
## ?p ?o ?target_class ?target_class_type
WHERE {
    # ?target_class a owl:Restriction .
    ##?property a owl:ObjectProperty .  ## drastically increases runtime
    
    ?r a owl:Restriction; 
        owl:onProperty ?property;
        owl:someValuesFrom ?target_class.
    
    ?target_class a ?target_class_type .
    ?target_class ?p ?o.  ##explore
    ?o ?p_o ?oe.
    
    #?property rdfs:label ?property_label.
    
    #?target_class owl:onProperty ?on_orpoerty_1 ;
    #owl:someValuesFromv ?target_class_1.
    
    
    #OPTIONAL {
    #    ?on_orpoerty_1 rdfs:domain ?domain; rdfs:range ?range.
    #    ?domain rdfs:label ?domain_class_label .
    #    ?range rdfs:label ?range_class_label .
    #}
    #?target_class_1 rdfs:label ?target_class_label.
    ##FILTER(STRSTARTS(STR(?resource), "http://www.w3.org/2002/07/owl#"))
    FILTER(STRSTARTS(STR(?p), "http://www.w3.org/2002/07/owl#"))
}
order by (?p)
"""
data_returned =  model.execute(q)
DSA = {
    "count_result": count_result.get('result')
    , "data_returned": data_returned.get('result')
    , "dsq": dsq
}

    
       
       
open('dsq_results/05.json','w').write(json.dumps(DSA       , indent=2))
## print(json.dumps(DSA       , indent=2))



In [ ]:
dsq = "How many axioms involve object properties on which both domain and range have also been asserted?"

count  = """
SELECT (COUNT(DISTINCT ?r) as ?restriction_count) 
WHERE {
    ?target_class a owl:Class .
    ##?property a owl:ObjectProperty .  ## drastically increases runtime
    
    ?r a owl:Restriction; 
        owl:onProperty ?property;
        owl:someValuesFrom ?target_class.
    ?property rdfs:label ?property_label.
    
    #OPTIONAL {
        ?property rdfs:domain ?domain; rdfs:range ?range.
        ?domain rdfs:label ?domain_class_label .
        ?range rdfs:label ?range_class_label .
    #}
    ?target_class rdfs:label ?target_class_label.
}
"""
count_result = model.execute(count) 


q  = """
SELECT ?property_label ?domain_class_label ?range_class_label ?target_class_label
WHERE {
    ?target_class a owl:Class .
    ##?property a owl:ObjectProperty .  ## drastically increases runtime
    
    ?r a owl:Restriction; 
        owl:onProperty ?property;
        owl:someValuesFrom ?target_class.
    ?property rdfs:label ?property_label.
    
    #OPTIONAL {
        ?property rdfs:domain ?domain; rdfs:range ?range.
        ?domain rdfs:label ?domain_class_label .
        ?range rdfs:label ?range_class_label .
    #}
    ?target_class rdfs:label ?target_class_label.
}
"""
data_returned =  model.execute(q)
DSA = {
    "count_result": count_result.get('result')
    , "data_returned": data_returned.get('result')
    , "dsq": dsq
}

    
       
       
open('dsq_results/Q3.json','w').write(json.dumps(DSA       , indent=2))



In [ ]:
dsq = "Why are there so many namespaces in the CCO ontology?" 
complexity = 0

results = []
for pfx in model.cco_ns:
    
    q = f"""
    SELECT  ?ns (count(distinct ?resource) as ?count)
    WHERE {{
        ?resource a ?type.
        FILTER(STRSTARTS(STR(?resource), "{ model.cco_ns.get(pfx) }"))
    }}  
    """
    res = model.execute_get_resultset(q)
    for x in  res.get('results'):
        res = x.asdict()
        res['namespace'] = pfx
        results.append(res)
        
##open((json.dumps(results, indent=2))
dsa = {
    "dsq": dsq
    , "complexity": complexity
    , "data_result": results
    , "runtime": res.get('query_runtime_ms')
}
open("dsq_results/Q2.json",'w').write(json.dumps(dsa,indent=2))





In [ ]:
## Complexity = 1
## Of the namespaces that actually contain resources, what are the types and counts of each type?  

results = []
for pfx in model.cco_ns:
    
    q = f"""
    SELECT ?type ?ns (count(distinct ?resource) as ?count)
    WHERE {{
        ?resource a ?type.
        FILTER(STRSTARTS(STR(?resource), "{ model.cco_ns.get(pfx) }"))
    }}  GROUP BY ?type
    """
    res = model.execute_get_resultset(q).get('results')

    for x in res:
        res = x.asdict()
        res['namespace'] = pfx
        results.append(res)
        
open('dsq_results/Q1.json','w').write(json.dumps(results, indent=2))


In [ ]:
q  = """
SELECT (COUNT(DISTINCT ?r) as ?restriction_count) 
WHERE {
    ?r a owl:Restriction
}
"""
for c in model.execute(q):
    print(json.dumps(c.asdict()) )